# multilabel_classifier로 val playlist를 채워보자
## 1. val의 song을 기준으로 tag를 10개씩 채우기

In [ ]:
train = pd.read_json('datas/train.json')
val = pd.read_json('datas/val.json')
song_meta = pd.read_json('datas/song_meta.json')
raw_df = pd.read_csv('datas/df_for_MLC.csv')

### 1. fit, predict를 위한 train + val Dataframe 만들기
#### 연도의 원핫인코딩

In [10]:
raw_df[['issue_year','issue_month']] = raw_df[['issue_year','issue_month']].apply(lambda x : [str(i) for i in x])
final_df_year_month_dummies = pd.get_dummies(raw_df[['issue_year','issue_month']])
final_df_year_month_dummies.reset_index(inplace=True)
final_df_year_month_dummies.rename(columns={'index':'id'},inplace=True)
final_df_one_hot = raw_df[raw_df.columns.difference(['issue_year','issue_month'])].merge(final_df_year_month_dummies, on='id')
final_df_one_hot

NameError: name 'raw_df' is not defined

#### id 필터링

In [ ]:
final_df_no_id = final_df_one_hot[final_df_one_hot.columns.difference(['id'])]
final_df_no_id[['population','like_cnt','tag_score']]

#### 정규화

In [ ]:
from sklearn.preprocessing import minmax_scale
MinMax_scaled = minmax_scale(final_df_no_id, axis=0, copy=True, )
df_for_KNN = pd.DataFrame(MinMax_scaled, columns = final_df_no_id.columns)
df_for_KNN['id'] = final_df_one_hot['id']
df_for_KNN

#### train과 val 데이터 나누기

In [ ]:
# train data
df_songs = train['songs'].apply(lambda x : [str(i) for i in x])
df_songs = train['songs'].reset_index(drop=True)
unique_song_list = np.concatenate(df_songs)
unique_song_list = list(set(unique_song_list))
unique_song_list = [int(i) for i in unique_song_list]
df_song_train = df_for_KNN[df_for_KNN['id'].isin(unique_song_list)]
df_song_train.drop(columns = 'Unnamed: 0', inplace = True)
df_song_train

In [ ]:
# val data
val_songs = val['songs'].apply(lambda x : [str(i) for i in x])
val_songs = val['songs'].reset_index(drop=True)
unique_song_list = np.concatenate(val_songs)
unique_song_list = list(set(unique_song_list))
unique_song_list = [int(i) for i in unique_song_list]
df_song_val = df_for_KNN[df_for_KNN['id'].isin(unique_song_list)]
df_song_val.drop(columns = 'Unnamed: 0', inplace = True)
df_song_val

#### trian에 y(tag) 붙이기

In [ ]:
ls = []
for i in train[['tags','songs']].index:
#     tags = train.get_value(i,'tags')
#     songs = train.get_value(i,'songs')
    tags = train.at[i,'tags']
    songs = train.at[i,'songs']
    for song in songs:
        ls.append({'id':song,'tags': tags})
song_tag = pd.DataFrame(ls)
song_tag_df = song_tag.groupby('id').agg(sum).reset_index()
song_tag_df

In [ ]:
df_song_train = df_song_train.merge(song_tag_df,on='id')
df_song_train.tail()

#### 피클파일로 저장

In [ ]:
df_song_train.set_index('id', inplace = True)
df_song_val.set_index('id', inplace = True)

#### 불러오기 (확인용)

In [ ]:
df_song_train.to_pickle('datas/df_song_train.pkl')
df_song_val.to_pickle('datas/df_song_val.pkl')

### 2. keras model 생성 
- song -> tags

In [6]:
df_song_train = pd.read_pickle('datas/df_song_train.pkl')

#### train에 y를 생성

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer
raw_te_tags_mlb = MultiLabelBinarizer(sparse_output=True)
raw_te_tags = list(df_song_train.tags)
y = raw_te_tags_mlb.fit_transform(raw_te_tags)
y_classes = raw_te_tags_mlb.classes_

#### train의 X를 생성

In [8]:
X = df_song_train[df_song_train.columns.difference(['tags'])]
X = X.values

#### train의 song으로 tag를 예측하는 모델 생성
- multi_tag_classification

In [ ]:
# 모델 생성 및 하이퍼 파라미터 튜닝
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
from keras.optimizers import SGD

with K.tf_ops.device('/device:CPU:0'):
    model = Sequential()

    model.add(Dense(32, input_shape=X.shape[1:], activation='relu'))
    
    model.add(Dense(64, activation='relu'))

    model.add(Dense(256, activation='relu'))

    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(y.shape[1], activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam')
    
    model_dir = './model'

    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    model_path = model_dir + '/multi_tag_classification.h5'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='loss', verbose=1, save_weights_only=True, period=5)
    early_stopping = EarlyStopping(monitor='loss', patience=5)

In [ ]:
# 배치사이즈, 에포크, 검증 데이터 사이즈 수정이 필요해보임
history = model.fit(X, y, batch_size=1000, epochs=10, validation_split = 0.2, callbacks=[checkpoint, early_stopping])

In [ ]:
# 그래프
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['loss'])
plt.title('predict_tags_model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.show()

In [ ]:
## 모델 저장(1) -> 모델의 껍데기 fit되지않은애가 저장됨.
# model_json = model.to_json()
# with open("model/multi_tag_classification.json", "w") as json_file : 
#     json_file.write(model_json)

In [ ]:
# # 모델 불러오기(1) -> 모델의 껍데기 fit되지 않은애가 불러와짐
# from keras.models import model_from_json 
# # json_file = open("model/multi_tag_classification.json", "r") 
# loaded_model_json = json_file.read() 
# json_file.close() 
# model = model_from_json(loaded_model_json)

In [ ]:
# 모델 가중치 불러오기(가중치만 저장됐을떄) -> 'multi_tag_classification.h5'
# 위의 모델 정의부터 하고, 빈 모델에 가중치를 덮어 씌워주는식)
# 모델 생성 후 잘 저장하였다면 안써도 됨 결국 아래의 저장, 불러오기만 잘 하면됨.
# from keras.models import load_model
# model.load_weights('model/multi_tag_classification.h5')

In [ ]:
# 모델 저장(2) fit된모델이 저장됨
model.save('model/multi_tag_classification_last.h5')

In [ ]:
# 모델 불러오기 fit된 모델이 불러와짐
from keras.models import load_model
model = load_model('model/multi_tag_classification_last.h5')

# multilabel_classifier로 val playlist를 채워보자
## 2. val의 tag를 기준으로 songs를 100개씩 채우기

In [ ]:
# train = pd.read_json('datas/train.json')
# val = pd.read_json('datas/val.json')
# song_meta = pd.read_json('datas/song_meta.json')
# raw_df = pd.read_csv('datas/df_for_MLC.csv')

### 1. train + val Dataframe 생성 (predict 과정시 사용하기 위함)
#### tag에 장르 붙이기

In [ ]:
song_meta = pd.read_json('datas/song_meta.json')

In [ ]:
raw_datas = pd.concat([train, val]).reset_index(drop = True)
ls = []
for i in raw_datas[['tags','songs']].index:
    tags = raw_datas.at[i,'tags']
    songs = raw_datas.at[i,'songs']
    for song in songs:
        ls.append({'id':song,'tags': tags})
song_tag = pd.DataFrame(ls)

In [ ]:
song_tags = song_tag.groupby('id').agg(sum).reset_index()
song_tags

#### tag의 count 붙이기

In [ ]:
song_tags['tags_num'] = np.nan
for i in range(0,len(song_tags)):
    song_tags['tags_num'][i] = sorted(Counter(song_tags['tags'][i]).items(), key = lambda x:x[1],reverse=True)
    song_tags['tags_num'][i] = dict(song_tags['tags_num'][i])

In [ ]:
sample = song_meta[['id', 'song_gn_gnr_basket', 'song_gn_dtl_gnr_basket']]
song_grn = sample['song_gn_gnr_basket'] + sample['song_gn_dtl_gnr_basket']
song = pd.DataFrame(song_grn)
song.reset_index(inplace=True)
song = song.rename(columns={"index": "id", 0: 'gn'})
song

#### song_tags와 song을 merge (train + val의 노래, tag모두 포함)

In [ ]:
temp = pd.merge(song_tags, song, left_on = 'id', right_on= 'id')
temp

#### tag에 장르 붙이기(group by)
- 데이터가 많아서 한번에하면 오래걸려서 4번에 나누어서함

In [ ]:
from tqdm.notebook import tqdm
tag_gns = []
for i in tqdm(range(0, len(temp))):
    for j in range(0, len(temp['tags_num'][i])):
        tag_gns.append({'tags': list(temp['tags_num'][i].items())[j][0],
                        'gn': temp['gn'][i] * list(temp['tags_num'][i].items())[j][1]})

In [ ]:
tag_gns_df = pd.DataFrame(tag_gns)
tag_gns_df

In [ ]:
# df의 데이터가 너무많아서 4번으로 나눠서 함(1/4)
from collections import Counter
from tqdm.notebook import tqdm

empty_df = []
for i in tqdm(range(0,len(tag_gns_df),100000)):
    sample = tag_gns_df[i:i+100000]
    temp = pd.DataFrame(sample['gn'].groupby(sample['tags']).sum())
    temp.reset_index(inplace = True)
    empty_df.append(temp)
tag_gns_df = pd.concat(empty_df)
tag_gns_df

In [ ]:
# df의 데이터가 너무많아서 4번으로 나눠서 함(2/4)
empty_df = []
for i in tqdm(range(0,len(tag_gns_df),100000)):
    sample = tag_gns_df[i:i+100000]
    temp = pd.DataFrame(sample['gn'].groupby(sample['tags']).sum())
    temp.reset_index(inplace = True)
    empty_df.append(temp)
tag_gns_df = pd.concat(empty_df)
tag_gns_df

In [ ]:
# df의 데이터가 너무많아서 4번으로 나눠서 함(3/4)
empty_df = []
for i in tqdm(range(0,len(tag_gns_df),100000)):
    sample = tag_gns_df[i:i+100000]
    temp = pd.DataFrame(sample['gn'].groupby(sample['tags']).sum())
    temp.reset_index(inplace = True)
    empty_df.append(temp)
tag_gns_df = pd.concat(empty_df)
tag_gns_df

In [ ]:
# df의 데이터가 너무많아서 4번으로 나눠서 함(4/4)
tag_gns = pd.DataFrame(tag_gns_df['gn'].groupby(tag_gns_df['tags']).sum())
tag_gns.reset_index(inplace = True)
tag_gns

#### tag count 붙이기

In [ ]:
from collections import Counter
tag_gns['gn_num'] = np.nan
for i in range(0,len(tag_gns)):
    tag_gns['gn_num'][i] = sorted(Counter(tag_gns['gn'][i]).items(), key = lambda x:x[1],reverse=True)
    tag_gns['gn_num'][i] = dict(tag_gns['gn_num'][i])
tag_gns

#### tag별 장르 구분
- tag에 언급된 노래들의 장르 count

In [ ]:
genre = pd.read_json('./datas/genre_gn_all.json', typ = 'seriese')
all_genre = list(genre.index)

In [ ]:
# 이코드가 오래걸립니다. 약 1시간쯤..?
from tqdm.notebook import tqdm
for i in tqdm(tag_gns.index):
    count = pd.DataFrame(list(tag_gns.loc[i]['gn_num'].items()), columns=['gn', i]).set_index('gn').T
    tag_gns.update(count)

#### 정규화
- tag_gns에서 필요없는 컬럼 버리고 정규화 시킴

In [ ]:
temp = tag_gns['tags']
tag_gns.drop(columns = ['gn','gn_num','tags'], inplace = True)

In [ ]:
from sklearn.preprocessing import minmax_scale
MinMax_scaled = minmax_scale(tag_gns, axis=0, copy=True, )
tag_gns_scaled = pd.DataFrame(MinMax_scaled, columns = tag_gns.columns)
tag_gns_scaled['tags'] = temp
tag_gns_scaled

#### df_tags를 train에 있는 tag와 val에 있는 tag로 나누어야함.

In [ ]:
# train 데이터
df_tag = train['tags'].apply(lambda x : [str(i) for i in x])
df_tag = train['tags'].reset_index(drop=True)
unique_tas_list = np.concatenate(df_tag)
unique_tas_list = list(set(unique_tas_list))
unique_tas_list = [str(i) for i in unique_tas_list]
df_tag_train = tag_gns_scaled[tag_gns_scaled['tags'].isin(unique_tas_list)]
df_tag_train

In [ ]:
# val 데이터
df_tag = val['tags'].apply(lambda x : [str(i) for i in x])
df_tag = val['tags'].reset_index(drop=True)
unique_tas_list = np.concatenate(df_tag)
unique_tas_list = list(set(unique_tas_list))
unique_tas_list = [str(i) for i in unique_tas_list]
df_tags_val = tag_gns_scaled[tag_gns_scaled['tags'].isin(unique_tas_list)]
df_tags_val

#### train에 y(song) 붙이기
- 이것도 너무많아서 6번 나눠서함, 사실 for문 돌려도되는데.. 귀찮아요..

In [ ]:
from tqdm.notebook import tqdm
ls = []
for i in tqdm(train[['tags','songs']].index):
    tags = train.at[i,'tags']
    songs = train.at[i,'songs']
    for tag in tags:
        ls.append({'tags':tag,'songs': songs})
tag_song = pd.DataFrame(ls)
tag_song

In [ ]:
empty_df = []
for i in tqdm(range(0,len(tag_song),10000)):
    sample = tag_song[i:i+10000]
    tag_songs = pd.DataFrame(sample['songs'].groupby(sample['tags']).sum())
    tag_songs.reset_index(inplace = True)
    empty_df.append(tag_songs)
df = pd.concat(empty_df)
df

In [ ]:
empty_df = []
for i in tqdm(range(0,len(tag_song),10000)):
    sample = df[i:i+10000]
    tag_songs = pd.DataFrame(sample['songs'].groupby(sample['tags']).sum())
    tag_songs.reset_index(inplace = True)
    empty_df.append(tag_songs)
df = pd.concat(empty_df)
df

In [ ]:
empty_df = []
for i in tqdm(range(0,len(tag_song),10000)):
    sample = df[i:i+10000]
    tag_songs = pd.DataFrame(sample['songs'].groupby(sample['tags']).sum())
    tag_songs.reset_index(inplace = True)
    empty_df.append(tag_songs)
df = pd.concat(empty_df)
df

In [ ]:
empty_df = []
for i in tqdm(range(0,len(tag_song),10000)):
    sample = df[i:i+10000]
    tag_songs = pd.DataFrame(sample['songs'].groupby(sample['tags']).sum())
    tag_songs.reset_index(inplace = True)
    empty_df.append(tag_songs)
df = pd.concat(empty_df)
df

In [ ]:
tag_songs = pd.DataFrame(df['songs'].groupby(df['tags']).sum())
tag_songs.reset_index(inplace = True)
tag_songs

In [ ]:
# y를 붙임
df_tags_train = pd.merge(df_tag_train, tag_songs, on = 'tags')
df_tags_train

#### 저장하기

In [ ]:
# tag를 인덱스로 저장
df_tags_train.set_index('tags', inplace = True)
df_tags_val.set_index('tags', inplace = True)

#### 저장

In [ ]:
# 저장
df_tags_train.to_pickle('datas/df_tags_train.pkl')
df_tags_val.to_pickle('datas/df_tags_val.pkl')

### 2. keras model 생성 
- tag -> songs

#### 불러오기

In [1]:
df_tags_train = pd.read_pickle('datas/df_tags_train.pkl')
df_tags_val = pd.read_pickle('datas/df_tags_val.pkl')

In [2]:
from sklearn.preprocessing import MultiLabelBinarizer
raw_te_tags_mlb = MultiLabelBinarizer(sparse_output=True)
raw_te_tags = list(df_tags_train.songs)
y = raw_te_tags_mlb.fit_transform(raw_te_tags)
y_classes = raw_te_tags_mlb.classes_

In [3]:
X = df_tags_train[df_tags_train.columns.difference(['songs'])]
X = X.values

#### train의 tag에서 song을 예측하는 모델 생성
- multi_song_classification

In [8]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
from keras.optimizers import SGD

with K.tf_ops.device('/device:CPU:0'):
    model = Sequential()

    model.add(Dense(32, input_shape=X.shape[1:], activation='relu'))
    
    model.add(Dense(64, activation='relu'))

    model.add(Dense(256, activation='relu'))

    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(y.shape[1], activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam')
    
    model_dir = './model'

    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    model_path = model_dir + '/multi_song_classification.h5'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='loss', verbose=1, save_weights_only=True, period=5)
    early_stopping = EarlyStopping(monitor='loss', patience=5)

In [9]:
history = model.fit(X, y, batch_size=100, epochs=10, validation_split = 0.2, callbacks=[checkpoint, early_stopping])

Train on 23328 samples, validate on 5832 samples
Epoch 1/10
  100/23328 [..............................] - ETA: 9:22 - loss: 0.6894

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['loss'])
plt.title('predict_songs_model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.show()

In [ ]:
## 모델 저장(1) -> 모델의 껍데기 fit되지않은애가 저장됨.
# model_json = model.to_json()
# with open("model/multi_tag_classification.json", "w") as json_file : 
#     json_file.write(model_json)

In [ ]:
# # 모델 불러오기(1) -> 모델의 껍데기 fit되지 않은애가 불러와짐
# from keras.models import model_from_json 
# # json_file = open("model/multi_tag_classification.json", "r") 
# loaded_model_json = json_file.read() 
# json_file.close() 
# model = model_from_json(loaded_model_json)

In [ ]:
# 모델 가중치 불러오기(가중치만 저장됐을떄) -> 'multi_tag_classification.h5'
# 위의 모델 정의부터 하고, 빈 모델에 가중치를 덮어 씌워주는식)
# 모델 생성 후 잘 저장하였다면 안써도 됨 결국 아래의 저장, 불러오기만 잘 하면됨.
# from keras.models import load_model
# model.load_weights('model/multi_tag_classification.h5')

In [ ]:
# 모델 저장(2)
model.save('model/multi_song_classification_last.h5')

In [ ]:
# 모델 불러오기(2)
from keras.models import load_model
load_model = load_model('model/multi_song_classification_last.h5')